In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import truncnorm
from fcit import fcit
from tqdm import tqdm

fcit: small pvlaue means dependant, large means independant.

In [2]:
def radius(A): # consistently fastest
    return np.max(np.abs(np.linalg.eigvals(A)))

def find_stationary(e,c):
    y,x,z,w = e[0].copy(), e[1].copy(), e[2].copy(), e[3].copy()
    assert np.all(x*y<c), ( (x*y)[x*y>c][:2], (x)[x*y>c][:2], (y)[x*y>c][:2] )
    assert np.all(1-x*y>1-c)
    Z = (w*y+z)/(1-x*y)
    W = (z*x+w)/(1-x*y)
    return np.vstack([y, x,Z,W])

In [4]:
c = 0.5
num_points = 100000
# num_points = 10000

Pvals_intrinsic = []
for iteration in tqdm(range(100)):

    # Monte Carlo from P(V)
    mu = np.zeros(4)
    cov = np.diag(np.ones(4))
    e = np.random.multivariate_normal(mu,cov,num_points).T

    e_mask = np.all(np.abs(e)<c,axis=0)
    e = e[:,e_mask]
    print(e.shape)
    assert not np.any(e>c)

    Pv = find_stationary(e,c)
    if c < 1.0:
        assert not np.any(Pv>1.0)
    # for vi in Pv:
    #     plt.hist(vi,bins=100)
    #     plt.show()


    # Test whether ( V[0], V[1] | V[2], V[3] ) is independent, or dependant.

    x = Pv[0,:].reshape((e.shape[1],1))
    y = Pv[1,:].reshape((e.shape[1],1))
    z = Pv.T[:,2:]
    print(x.shape,y.shape,z.shape)

    pval = fcit.test(x, y, z)
    print("Pval:",pval)

    if pval > 0.1:
        print("Independant")
    else:
        print("Dependant")
    
    Pvals_intrinsic.append(pval)

  0%|          | 0/100 [00:00<?, ?it/s]

(4, 2157)
(2157, 1) (2157, 1) (2157, 2)


  1%|          | 1/100 [00:04<07:54,  4.79s/it]

Pval: 0.19886705313412473
Independant
(4, 2115)
(2115, 1) (2115, 1) (2115, 2)


  2%|▏         | 2/100 [00:09<07:57,  4.87s/it]

Pval: 0.30565342555544683
Independant
(4, 2044)
(2044, 1) (2044, 1) (2044, 2)


  3%|▎         | 3/100 [00:14<07:59,  4.94s/it]

Pval: 0.18549674286799436
Independant
(4, 2190)
(2190, 1) (2190, 1) (2190, 2)


  3%|▎         | 3/100 [00:19<10:19,  6.38s/it]


KeyboardInterrupt: 

In [ ]:
plt.hist(Pvals_intrinsic)
plt.show()

In [ ]:
raise ValueError

In [ ]:
c = 100.
num_points = 2200

Pvals_counter = []
for iteration in tqdm(range(100)):

    # Monte Carlo from P(V)
    mu = np.zeros(4)
    cov = np.diag(np.ones(4))
    e = np.random.multivariate_normal(mu,cov,num_points).T

    e_mask = np.all(np.abs(e)<c,axis=0)
    e = e[:,e_mask]
    print(e.shape)
    assert not np.any(e>c)

    Pv = find_stationary(e,c)
    if c < 1.0:
        assert not np.any(Pv>1.0)
    # for vi in Pv:
    #     plt.hist(vi,bins=100)
    #     plt.show()


    # Test whether ( V[0], V[1] | V[2], V[3] ) is independent, or dependant.

    x = Pv[0,:].reshape((e.shape[1],1))
    y = Pv[1,:].reshape((e.shape[1],1))
    z = Pv.T[:,2:]
    print(x.shape,y.shape,z.shape)

    pval = fcit.test(x, y, z)
    print("Pval:",pval)

    if pval > 0.1:
        print("Independant")
    else:
        print("Dependant")
    
    Pvals_counter.append(pval)

In [ ]:
plt.hist(Pvals_counter)
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 22})

plt.figure(figsize=(9, 3))
plt.hist(Pvals_counter,label="original",alpha=0.5,color='r')
plt.hist(Pvals_intrinsic,label="intrinsic",alpha=0.5,color='g')
plt.axis([0, 1, 0, 90])
plt.xlabel("P-value (fcit)")
plt.ylabel("frequency")
plt.legend()
plt.title("$(V_1 \perp \!\!\! \perp V_2 | V_3, V_4)$ ?")
# plt.show()
plt.savefig("counterexample_domain_restriction.png")

In [ ]:
plt.rcParams.update({'font.size': 22})

plt.figure(figsize=(9, 3))
plt.hist(Pvals_counter,label="original",alpha=0.5,color='r')
# plt.hist(Pvals_intrinsic,label="intrinsic",alpha=0.5,color='g')
plt.axis([0, 1, 0, 90])
plt.xlabel("P-value (fcit)")
plt.ylabel("frequency")
# plt.legend()
plt.title("$(V_1 \perp \!\!\! \perp V_2 | V_3, V_4)$ ?")
# plt.show()
plt.savefig("counterexample.png")